In [1]:
import pandas as pd
from pathlib import Path
from datetime import datetime 

# Mostra todas as colunas, sem limite
pd.set_option('display.max_columns', None)

# (Opcional) Ajustes que costumam ajudar:
pd.set_option('display.expand_frame_repr', False)   # evita quebra de linha
pd.set_option('display.max_colwidth', None)         # mostra o conteúdo completo das células
pd.set_option('display.width', 0)  

# Pasta com os arquivos Parquet
pasta = Path("/Users/silmara.barnabe/Desktop/Projects/vigimed/data/02_bronze/notificacoes")
layer ='03_silver'
dataset = 'notificacoes'
ts = datetime.now().strftime("%Y_%m_%d_%H_%M_%S") #'2025_06_30_14_17_00'
output = "/Users/silmara.barnabe/Desktop/Projects/vigimed/data/"+layer+"/"+dataset+"/"+ts+".parquet"
print(output) 

/Users/silmara.barnabe/Desktop/Projects/vigimed/data/03_silver/notificacoes/2025_08_14_16_18_49.parquet


# dates

In [2]:
# Lista todos os arquivos parquet
arquivos = list(pasta.glob("*.parquet"))

dfs = []

for arquivo in arquivos:
    try:
        df_temp = pd.read_parquet(arquivo)
        df_temp['arquivo'] = arquivo.name  # cria coluna com nome do arquivo
        dfs.append(df_temp)
    except Exception as e:
        print(f"Erro ao ler {arquivo.name}: {e}")

# Concatena todos os dataframes lidos
if dfs:
    df_final = pd.concat(dfs, ignore_index=True)
    print("DataFrames concatenados com sucesso!")
else:
    print("Nenhum arquivo foi carregado.")

# Mostra as primeiras linhas
df_final.head()


DataFrames concatenados com sucesso!


,UF,TIPO_ENTRADA_VIGIMED,RECEBIDO_DE,IDENTIFICACAO_NOTIFICACAO,DATA_INCLUSAO_SISTEMA,DATA_ULTIMA_ATUALIZACAO,DATA_NOTIFICACAO,TIPO_NOTIFICACAO,NOTIFICACAO_PARENT_CHILD,DATA_NASCIMENTO,IDADE_MOMENTO_REACAO,GRUPO_IDADE,IDADE_GESTACIONAL_MOMENTO_REACAO,SEXO,GESTANTE,LACTANTE,PESO_KG,ALTURA_CM,REACAO_EVENTO_ADVERSO_MEDDRA,GRAVE,GRAVIDADE,DESFECHO,DATA_INICIO_HORA,DATA_FINAL_HORA,DURACAO,RELACAO_MEDICAMENTO_EVENTO,NOME_MEDICAMENTO_WHODRUG,ACAO_ADOTADA,NOTIFICADOR,arquivo
0,SP,Empresas Farmacêuticas,Empresa Farmacêutica,BR-ANVISA-300212656,20230928,20230928,None,Notificação espontânea,None,19900131,30 ano,None,None,Feminino,Não,Não,68.0,165,Hemiparesia,Sim,Outro efeito clinicamente significativo,Recuperado,20210125,20210206,12 dia,Concomitante,Tamisa,Não aplicável,Médico,2025_08_14_11_22_54.parquet
1,SP,Empresas Farmacêuticas,Empresa Farmacêutica,BR-ANVISA-300208322,20230901,20230901,None,Notificação espontânea,None,None,None,None,None,Feminino,Não,Não,None,None,Cefaleia,Não,Outro efeito clinicamente significativo,Desconhecido,20210122,None,None,Suspeito,CoronaVac,Não aplicável,Consumidor ou outro não profissional de saúde,2025_08_14_11_22_54.parquet
2,SP,Empresas Farmacêuticas,Empresa Farmacêutica,BR-ANVISA-300214015,20231006,20231006,None,Notificação espontânea,None,19710522,49 ano,None,None,Masculino,Não,Não,None,None,Nefrolitíase,Sim,Hospitalização,Desconhecido,20210203,None,None,Concomitante,Metformina,None,Médico,2025_08_14_11_22_54.parquet
3,SP,Empresas Farmacêuticas,Empresa Farmacêutica,BR-ANVISA-300212385,20230927,20230927,None,Notificação espontânea,None,None,None,None,None,Masculino,Não,Não,None,None,Abscesso nasal,Sim,Outro efeito clinicamente significativo,Desconhecido,None,None,None,Suspeito,Vacina adsorvida COVID-19 (inativada),None,Consumidor ou outro não profissional de saúde,2025_08_14_11_22_54.parquet
4,SP,Empresas Farmacêuticas,Empresa Farmacêutica,BR-ANVISA-300212764,20230929,20230929,None,Notificação espontânea,None,19820505,39 ano,None,None,Masculino,Não,Não,None,None,Descompensação metabólica diabética,Sim,Outro efeito clinicamente significativo,Desconhecido,None,None,None,Suspeito,CoronaVac,Desconhecido,Consumidor ou outro não profissional de saúde,2025_08_14_11_22_54.parquet


# Summary

In [3]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1468104 entries, 0 to 1468103
Data columns (total 30 columns):
 #   Column                            Non-Null Count    Dtype 
---  ------                            --------------    ----- 
 0   UF                                1035035 non-null  object
 1   TIPO_ENTRADA_VIGIMED              1467711 non-null  object
 2   RECEBIDO_DE                       1214001 non-null  object
 3   IDENTIFICACAO_NOTIFICACAO         1468104 non-null  object
 4   DATA_INCLUSAO_SISTEMA             1468104 non-null  object
 5   DATA_ULTIMA_ATUALIZACAO           1467778 non-null  object
 6   DATA_NOTIFICACAO                  769923 non-null   object
 7   TIPO_NOTIFICACAO                  1468104 non-null  object
 8   NOTIFICACAO_PARENT_CHILD          5336 non-null     object
 9   DATA_NASCIMENTO                   1088368 non-null  object
 10  IDADE_MOMENTO_REACAO              926352 non-null   object
 11  GRUPO_IDADE                       588504 non-null 

In [4]:
def resumo_distintos_colunas(df: pd.DataFrame, cols=None) -> pd.DataFrame:
    """
    Retorna um resumo por coluna com:
    - n_registros (total de linhas)
    - n_distintos
    - %_distintos
    - n_nulos
    - %_nulos
    """
    if cols is None:
        cols = df.columns.tolist()
    df2 = df[cols]
    total = len(df2)

    nunique = df2.nunique(dropna=True)
    nnull = df2.isna().sum()

    out = (
        pd.DataFrame({
            "coluna": nunique.index,
            "n_registros": total,
            "n_distintos": nunique.values,
            "%_distintos": (nunique.values / total * 100) if total else 0.0,
            "n_nulos": nnull.values,
            "%_nulos": (nnull.values / total * 100) if total else 0.0,
        })
        .sort_values("%_distintos", ascending=False)
        .reset_index(drop=True)
    )

    if total:
        out["%_distintos"] = out["%_distintos"].round(2)
        out["%_nulos"] = out["%_nulos"].round(2)

    return out


def resumo_distintos_linhas(df: pd.DataFrame) -> pd.Series:
    """
    Retorna um resumo do dataset considerando todas as colunas:
    - total_linhas
    - linhas_distintas
    - %_linhas_distintas
    - linhas_duplicadas
    - %_linhas_duplicadas
    """
    total = len(df)
    distintos = len(df.drop_duplicates())
    duplicados = total - distintos

    pct_distintos = (distintos / total * 100) if total else 0.0
    pct_duplicados = (duplicados / total * 100) if total else 0.0

    return pd.Series({
        "total_linhas": total,
        "linhas_distintas": distintos,
        "%_linhas_distintas": round(pct_distintos, 2),
        "linhas_duplicadas": duplicados,
        "%_linhas_duplicadas": round(pct_duplicados, 2),
    })



In [6]:

# --- USO ---
cols = df_final.columns.tolist()

summary_cols = resumo_distintos_colunas(df_final, cols=cols)
summary_rows = resumo_distintos_linhas(df_final)
 

In [7]:
summary_cols

,coluna,n_registros,n_distintos,%_distintos,n_nulos,%_nulos
0,IDENTIFICACAO_NOTIFICACAO,1468104,276940,18.86,0,0.00
1,DATA_NASCIMENTO,1468104,34301,2.34,379736,25.87
2,DATA_INICIO_HORA,1468104,25440,1.73,392310,26.72
3,DATA_FINAL_HORA,1468104,20192,1.38,723841,49.30
4,REACAO_EVENTO_ADVERSO_MEDDRA,1468104,13916,0.95,1572,0.11
5,NOME_MEDICAMENTO_WHODRUG,1468104,7631,0.52,2621,0.18
6,PESO_KG,1468104,4114,0.28,890084,60.63
7,DATA_NOTIFICACAO,1468104,2962,0.20,698181,47.56
8,DATA_ULTIMA_ATUALIZACAO,1468104,2320,0.16,326,0.02
9,DATA_INCLUSAO_SISTEMA,1468104,2319,0.16,0,0.00


In [9]:
print("\nResumo por linhas do dataset:\n", summary_rows)


Resumo por linhas do dataset:
 total_linhas           1468104.00
linhas_distintas        553689.00
%_linhas_distintas          37.71
linhas_duplicadas       914415.00
%_linhas_duplicadas         62.29
dtype: float64


# NOTIFICACAO_PARENT_CHILD

In [21]:
df_final.NOTIFICACAO_PARENT_CHILD.value_counts()

NOTIFICACAO_PARENT_CHILD
Sim    5336
Name: count, dtype: int64

# SEXO

In [22]:
df_final.SEXO.value_counts()

SEXO
Feminino        879150
Masculino       530267
Desconhecido      5356
Name: count, dtype: int64

# GESTANTE 

In [23]:
df_final.GESTANTE.value_counts()

GESTANTE
Não     464872
Năo     114639
Sim       7604
Name: count, dtype: int64

# LACTANTE

In [24]:
df_final.LACTANTE.value_counts()

LACTANTE
Não     464564
Năo     114562
Sim       8009
Name: count, dtype: int64

In [25]:
df_final.LACTANTE.unique()

array(['Não ', None, 'Sim', 'Năo '], dtype=object)

# DATA_INICIO_HORA

In [ ]:
df_final.DATA_INICIO_HORA.value_counts().sort_index(ascending=False).head(10)

DATA_INICIO_HORA
22:18:00              5
20290611 23:00        6
20251028 09:10:00     5
20251010 14:00:00     5
20250627              5
20250607              5
20250510             10
20250509             15
20250509             10
20250508 12:20:10     5
Name: count, dtype: int64

In [11]:
df_final.DATA_FINAL_HORA.value_counts().sort_index(ascending=False).head(10)

DATA_FINAL_HORA
23:10:00             5
22:00:00             5
21:30                6
20420809             5
20260529             5
20251031 23:30:00    5
20251028 09:55:00    5
20251010 17:00:00    5
20250607             5
20250511             5
Name: count, dtype: int64

# RELACAO_MEDICAMENTO_EVENTO

In [17]:
df_final.RELACAO_MEDICAMENTO_EVENTO.value_counts()


RELACAO_MEDICAMENTO_EVENTO
Suspeito                        1419862
Medicamento não administrado      28159
Concomitante                       9619
Medicamento năo administrado       7029
Interação                          2771
Interaçăo                           659
Name: count, dtype: int64

# GRAVE

In [13]:
df_final.GRAVE.value_counts()


GRAVE
Sim    652527
Não    386559
Năo     90745
Name: count, dtype: int64

In [14]:
df_final.GRAVE.unique()

array(['Sim', 'Não', None, 'Năo'], dtype=object)

# DESFECHO

In [15]:
df_final.DESFECHO.value_counts().sort_index(ascending=False)

DESFECHO
Recuperado        687892
Năo Recuperado     19880
Não Recuperado     85583
Fatal              52241
Em recuperaçăo     25507
Em recuperação    111163
Desconhecido      246672
Name: count, dtype: int64

# SOC

In [28]:
df_final.SOC.value_counts()

AttributeError: 'DataFrame' object has no attribute 'SOC'

# GRAVIDADE

In [27]:
df_final.GRAVIDADE.value_counts()

GRAVIDADE
Outro efeito clinicamente significativo        395645
Hospitalização                                 121920
Resultou em óbito                               52953
Ameaça à vida                                   37888
Incapacidade persistente ou significativa       31063
Hospitalizaçăo                                  28746
Ameaça ŕ vida                                    8756
Anomalia congênita ou malformação ao nascer       583
Anomalia congęnita ou malformaçăo ao nascer       136
Name: count, dtype: int64